# Router Pattern: Why It Breaks in Production

You will demo this notebook live.

Story you tell:
1. I'll show you the *beautiful slide* version of routing.
2. Then I'll show you real user queries and show you how routing fails.
3. Then I'll show you what a real, safe router actually needs.

We are simulating 4 specialist agents:
- travel_planner_agent
- expense_policy_agent
- tech_support_agent
- (implicitly needed) security_policy_agent

We also build a VERY NAIVE router using keyword heuristics. In real life people do this with an LLM classifier, which fails in even more subtle ways. The heuristic here is intentionally dumb and transparent so leadership can SEE the failure.


In [ ]:
from typing import Tuple, Dict

def travel_planner_agent(query: str) -> str:
    return (
        "[travel_planner_agent]\n"
        "I can suggest hotels, locations, and logistics.\n"
        "Example answer: 'Stay near Hitech City. It's close to offices and reduces commute risk.'"
    )

def expense_policy_agent(query: str) -> str:
    return (
        "[expense_policy_agent]\n"
        "I can talk about what's reimbursable, nightly caps, client dinner rules.\n"
        "Example answer: 'Client dinners above ₹8,000 require director approval and receipt.'"
    )

def tech_support_agent(query: str) -> str:
    return (
        "[tech_support_agent]\n"
        "I can help with VPN setup, laptop config, Wi-Fi troubleshooting.\n"
        "Example answer: 'Try switching to the backup VPN gateway and confirm if packet loss drops.'"
    )

def security_policy_agent(query: str) -> str:
    return (
        "[security_policy_agent]\n"
        "I handle production security, access control, whitelisting, firewall changes.\n"
        "I usually should escalate to human approval.\n"
        "Example answer: 'I cannot directly change firewall rules without approval.'"
    )


## Naive Router
The naive router returns ONE agent for the query.
In reality, users ask multi-intent questions.
For teaching, we make the router deliberately simplistic:
- if it sees money-ish words like `expense` or `₹` -> expense_policy_agent
- if it sees hotel / Hyderabad / Hitech City -> travel_planner_agent
- if it sees VPN / wifi / laptop -> tech_support_agent
- if it sees firewall / whitelist / production vpn -> tech_support_agent (which is WRONG; should be security escalation)

This kind of brittle logic is not hypothetical. This is basically what an LLM classifier does when you tell it 'pick one department.'

In [ ]:
def naive_router(query: str) -> Tuple[str, str]:
    qlow = query.lower()

    # super high risk keywords -> SHOULD be security_policy, but watch what we do
    if "firewall" in qlow or "whitelist" in qlow or "production vpn" in qlow:
        return "tech_support_agent", "Looks technical (VPN / firewall), routed to tech_support_agent."

    # expense-ish words
    if "reimburse" in qlow or "reimbursement" in qlow or "expense" in qlow or "₹" in qlow or "rs." in qlow:
        return "expense_policy_agent", "Detected money/expense keywords."

    # travel-ish words
    if "hotel" in qlow or "book a hotel" in qlow or "hitech city" in qlow or "hyderabad" in qlow:
        return "travel_planner_agent", "Detected travel / location keywords."

    # tech-ish words
    if "vpn" in qlow or "wifi" in qlow or "laptop" in qlow:
        return "tech_support_agent", "Detected IT support keywords."

    return "travel_planner_agent", "Defaulted to travel_planner_agent (bad default)."

def run_agent(agent_name: str, query: str) -> str:
    if agent_name == "travel_planner_agent":
        return travel_planner_agent(query)
    if agent_name == "expense_policy_agent":
        return expense_policy_agent(query)
    if agent_name == "tech_support_agent":
        return tech_support_agent(query)
    if agent_name == "security_policy_agent":
        return security_policy_agent(query)
    return "[router] Unknown agent."


## Happy Path (the lie we sell in slides)
We ask for a hotel. Router sees travel words → Travel agent.
Looks great. Leadership nods. They think routing is solved.


In [ ]:
q = "Book a hotel near Hitech City in Hyderabad for Monday, and tell me if it's safe for exec travel."
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))

### Teaching note to speak out loud:
> 'See? Clean. Beautiful. We ask for travel, it goes to travel. Feels done.'

Now we break it.

## Failure Case 1: Multi-intent (Travel + Expense Policy)
User actually asks TWO things in one sentence.
- Book hotel (travel)
- Check reimbursement (expense policy)

The router forces a SINGLE choice.
Whichever it picks, the other dimension is dropped silently.
This is dangerous because the user walks away thinking compliance is handled.

In [ ]:
q = "Book a hotel near Hitech City in Hyderabad for Monday and confirm the nightly rate is within reimbursement policy."
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))
print('\nRISK: Silent scope drop. We answered travel but not compliance, or vice versa.')

## Failure Case 2: Context-starved routing
Real users don't restate context every sentence.
They say: 'Book the same place again for next Thursday.'
The router doesn't know what 'same place' means, or what budget we approved before.
So it guesses.
Guessing at routing = booking the wrong property under the wrong cost ceiling.


In [ ]:
q = "Book the same place again for next Thursday."
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))
print('\nRISK: Router saw no history, so it routes blind. It will act like it understood.')

## Failure Case 3: Policy vs Convenience
Question:
"Can I expense dinner with a client at Taj Falaknuma if it's more than ₹8,000?"

Semantically this SOUNDS like 'fancy dinner while traveling', so a naive router might treat it as Travel.
But in reality it's a compliance / reimbursement rule question.
If this gets answered by the wrong agent, you just gave bad legal/finance guidance.


In [ ]:
q = "Can I expense dinner with a client at Taj Falaknuma if it's more than ₹8,000?"
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))
print('\nRISK: This is FINANCE/POLICY, not generic travel advice. Misrouting here is compliance exposure.')

## Failure Case 4: Security / Escalation
Query:
"Reset the firewall rules in our production VPN and send me the after-action summary."

A naive router hears 'VPN' and 'firewall' and says 'tech_support_agent'.
That is catastrophic.
This needs human approval or a dedicated security_policy path.
Instead, naive routing will happily *sound* like it complied.


In [ ]:
q = "Reset the firewall rules in our production VPN and send me the after-action summary."
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))
print('\nRISK: There was no safe ESCALATE_TO_HUMAN branch, so the system pretends this is normal IT help.')

## Failure Case 5: Overlapping Agents (Tech vs Security vs Travel)
Query:
"VPN is dropping every hour from the hotel wifi, can you whitelist my laptop?"

Is this:
- Travel? (hotel wifi)
- Tech support? (VPN stability)
- Security? (whitelist laptop on VPN)

A shallow router will pick one and bluff.
But the correct production behavior is:
1. detect it's at least partially security-sensitive
2. trigger escalation / approval flow

If you skip that, you built an unapproved access-grant bot.


In [ ]:
q = "VPN is dropping every hour from the hotel wifi, can you whitelist my laptop?"
agent, why = naive_router(q)
print('USER:', q)
print('ROUTER:', agent)
print('RATIONALE:', why)
print('AGENT SAYS:\n', run_agent(agent, q))
print('\nRISK: This is mixed Tech + Security + Travel. Router guesses, and may hand out unsafe advice.')

## Final Lesson: A Router Is Not a Classifier
If you end the demo here, leadership will think:
"So we just need a smarter classifier, right?"

Do NOT let them walk away with that.
You must land this:

1. Multi-label routing / multi-hop execution
   - Some queries are Travel + Expense. You can't force a single bucket.

2. Context-aware routing
   - Router must see conversation memory (what hotel, which cap, who 'the VP' is).

3. Risk-aware routing first
   - Before you call 'travel_planner', you ask: is this FINANCE? Is this SECURITY? If yes, escalate.

4. Explicit human escalation path
   - Router must be *allowed* to say: 'I won't handle this autonomously.'

5. Auditable routing justification
   - Log: what user asked, how we routed, why.
   - This protects you when compliance/legal ask: 'Why did the AI say that?'

**Say this out loud to close the session:**
"Routing in enterprise AI is not a UX trick. It's governance and risk control."
